### Using the Census API to get zip-code level population, median income, pct_below_poverty and racial data for NYC zip codes.

In [48]:


# !pip install censusgeocode



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.3 MB/s eta 0:00:00


In [49]:
import json
import requests
from census import Census
from us import states
from us import states
import glob
from pprint import pprint
import censusgeocode as cg
from concurrent.futures import ThreadPoolExecutor
from tqdm.notebook import tqdm
import pandas as pd


In [2]:
# load keys
with open('keys.json') as f:
    keys = json.load(f)

# create census object
c = Census(keys['census_api_key'])

In [3]:
# define a function to get data from a particular table
def get_from_census(table_id, year):
    response = requests.get(f"https://api.census.gov/data/{year}/acs/acs5?get=NAME,group({table_id})&for=zip%20code%20tabulation%20area:*&key={keys['census_api_key']}")
    data = response.json()
    df = pd.DataFrame(data, columns=data[0]).drop(0)
    return df

# Race

In [4]:
# get race data for 2021
race_2021 = get_from_census("B02001", "2021")
race_2021


,NAME,B02001_001E,B02001_001EA,B02001_001M,B02001_001MA,B02001_002E,B02001_002EA,B02001_002M,B02001_002MA,B02001_003E,...,B02001_009EA,B02001_009M,B02001_009MA,B02001_010E,B02001_010EA,B02001_010M,B02001_010MA,GEO_ID,NAME,zip code tabulation area
1,ZCTA5 00601,17126,None,429,None,14463,None,626,None,243,...,None,323,None,111,None,127,None,860Z200US00601,ZCTA5 00601,00601
2,ZCTA5 00602,37895,None,279,None,23237,None,1340,None,688,...,None,629,None,10046,None,1186,None,860Z200US00602,ZCTA5 00602,00602
3,ZCTA5 00603,49136,None,841,None,36497,None,1337,None,1681,...,None,764,None,772,None,274,None,860Z200US00603,ZCTA5 00603,00603
4,ZCTA5 00606,5751,None,355,None,3319,None,479,None,27,...,None,160,None,18,None,30,None,860Z200US00606,ZCTA5 00606,00606
5,ZCTA5 00610,26153,None,382,None,15815,None,1081,None,642,...,None,573,None,6254,None,1022,None,860Z200US00610,ZCTA5 00610,00610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33770,ZCTA5 99923,13,None,20,None,13,None,20,None,0,...,None,10,None,0,None,10,None,860Z200US99923,ZCTA5 99923,99923
33771,ZCTA5 99925,917,None,118,None,411,None,78,None,4,...,None,10,None,153,None,41,None,860Z200US99925,ZCTA5 99925,99925
33772,ZCTA5 99926,1445,None,155,None,92,None,37,None,0,...,None,12,None,22,None,24,None,860Z200US99926,ZCTA5 99926,99926
33773,ZCTA5 99927,11,None,15,None,11,None,15,None,0,...,None,10,None,0,None,10,None,860Z200US99927,ZCTA5 99927,99927


In [5]:
# B02001_001E - Total
# B02001_002E - White alone
# B02001_003E - Black or African American alone
# B02001_004E - American Indian and Alaska Native alone
# B02001_005E - Asian alone
# B02001_006E - Native Hawaiian and Other Pacific Islander alone

In [6]:
# get racial data for 2021
racial_2021 = get_from_census("B02001", "2021")

# clean it up
racial_2021 = racial_2021[['zip code tabulation area', 'B02001_001E', 'B02001_002E', 'B02001_003E', 'B02001_004E', 'B02001_005E', 'B02001_006E']]
racial_2021.rename(columns={
    'zip code tabulation area': 'zip',
})

,zip,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E
1,00601,17126,14463,243,13,2,0
2,00602,37895,23237,688,44,46,0
3,00603,49136,36497,1681,54,38,0
4,00606,5751,3319,27,21,0,0
5,00610,26153,15815,642,0,0,0
...,...,...,...,...,...,...,...
33770,99923,13,13,0,0,0,0
33771,99925,917,411,4,335,7,0
33772,99926,1445,92,0,1304,6,0
33773,99927,11,11,0,0,0,0


In [7]:
racial_2021 = racial_2021.rename(columns={
    'zip code tabulation area': 'zip',
    'B02001_001E': 'total',
    'B02001_002E': 'white',
    'B02001_003E': 'black_or_african_american',
    'B02001_004E': 'american_indian_or_alaska_native',
    'B02001_005E': 'asian',
    'B02001_006E': 'native_hawaiian_and_other_pacific_islander'
})

In [8]:
# convert all columns to numeric
racial_2021 = racial_2021.apply(pd.to_numeric)


In [9]:
racial_2021.dtypes

zip                                           int64
total                                         int64
white                                         int64
black_or_african_american                     int64
american_indian_or_alaska_native              int64
asian                                         int64
native_hawaiian_and_other_pacific_islander    int64
dtype: object

In [10]:
# calculate percentages
racial_2021['pct_white'] = racial_2021['white'] / racial_2021['total'] * 100
racial_2021['pct_black_or_african_american'] = racial_2021['black_or_african_american'] / racial_2021['total'] * 100
racial_2021['pct_american_indian_or_alaska_native'] = racial_2021['american_indian_or_alaska_native'] / racial_2021['total'] * 100
racial_2021['pct_asian'] = racial_2021['asian'] / racial_2021['total'] * 100
racial_2021['pct_native_hawaiian_and_other_pacific_islander'] = racial_2021['native_hawaiian_and_other_pacific_islander'] / racial_2021['total'] * 100
racial_2021


,zip,total,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander,pct_white,pct_black_or_african_american,pct_american_indian_or_alaska_native,pct_asian,pct_native_hawaiian_and_other_pacific_islander
1,601,17126,14463,243,13,2,0,84.450543,1.418895,0.075908,0.011678,0.000000
2,602,37895,23237,688,44,46,0,61.319435,1.815543,0.116110,0.121388,0.000000
3,603,49136,36497,1681,54,38,0,74.277515,3.421117,0.109899,0.077336,0.000000
4,606,5751,3319,27,21,0,0,57.711702,0.469484,0.365154,0.000000,0.000000
5,610,26153,15815,642,0,0,0,60.471074,2.454785,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
33770,99923,13,13,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000
33771,99925,917,411,4,335,7,0,44.820065,0.436205,36.532170,0.763359,0.000000
33772,99926,1445,92,0,1304,6,0,6.366782,0.000000,90.242215,0.415225,0.000000
33773,99927,11,11,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000


In [11]:
racial_2021.dtypes



zip                                                 int64
total                                               int64
white                                               int64
black_or_african_american                           int64
american_indian_or_alaska_native                    int64
asian                                               int64
native_hawaiian_and_other_pacific_islander          int64
pct_white                                         float64
pct_black_or_african_american                     float64
pct_american_indian_or_alaska_native              float64
pct_asian                                         float64
pct_native_hawaiian_and_other_pacific_islander    float64
dtype: object

In [12]:
# get hispanic data for 2021
hispanic_2021 = get_from_census("B03002", "2021")
hispanic_2021


,NAME,B03002_001E,B03002_001EA,B03002_001M,B03002_001MA,B03002_002E,B03002_002EA,B03002_002M,B03002_002MA,B03002_003E,...,B03002_020EA,B03002_020M,B03002_020MA,B03002_021E,B03002_021EA,B03002_021M,B03002_021MA,GEO_ID,NAME,zip code tabulation area
1,ZCTA5 00601,17126,None,429,None,88,None,71,None,88,...,None,323,None,111,None,127,None,860Z200US00601,ZCTA5 00601,00601
2,ZCTA5 00602,37895,None,279,None,2246,None,442,None,1770,...,None,629,None,9714,None,1168,None,860Z200US00602,ZCTA5 00602,00602
3,ZCTA5 00603,49136,None,841,None,1015,None,395,None,731,...,None,765,None,678,None,246,None,860Z200US00603,ZCTA5 00603,00603
4,ZCTA5 00606,5751,None,355,None,41,None,52,None,41,...,None,160,None,18,None,30,None,860Z200US00606,ZCTA5 00606,00606
5,ZCTA5 00610,26153,None,382,None,1100,None,331,None,920,...,None,573,None,6233,None,1020,None,860Z200US00610,ZCTA5 00610,00610
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33770,ZCTA5 99923,13,None,20,None,13,None,20,None,13,...,None,10,None,0,None,10,None,860Z200US99923,ZCTA5 99923,99923
33771,ZCTA5 99925,917,None,118,None,879,None,116,None,392,...,None,10,None,10,None,12,None,860Z200US99925,ZCTA5 99925,99925
33772,ZCTA5 99926,1445,None,155,None,1398,None,146,None,89,...,None,12,None,0,None,10,None,860Z200US99926,ZCTA5 99926,99926
33773,ZCTA5 99927,11,None,15,None,11,None,15,None,11,...,None,10,None,0,None,10,None,860Z200US99927,ZCTA5 99927,99927


In [13]:
# B03002_001E - Total
# B03002_002E - Not Hispanic or Latino


In [14]:
hispanic_2021 = hispanic_2021[['zip code tabulation area', 'B03002_001E', 'B03002_002E']]
hispanic_2021 = hispanic_2021.rename(columns={
    'zip code tabulation area': 'zip',
})
hispanic_2021

,zip,B03002_001E,B03002_002E
1,00601,17126,88
2,00602,37895,2246
3,00603,49136,1015
4,00606,5751,41
5,00610,26153,1100
...,...,...,...
33770,99923,13,13
33771,99925,917,879
33772,99926,1445,1398
33773,99927,11,11


In [19]:
hispanic_2021 = hispanic_2021.rename(columns={
    'B03002_001E': 'total',
    'B03002_002E': 'not_hispanic_or_latino',
})

hispanic_2021

,zip,total,not_hispanic_or_latino
1,601,17126,88
2,602,37895,2246
3,603,49136,1015
4,606,5751,41
5,610,26153,1100
...,...,...,...
33770,99923,13,13
33771,99925,917,879
33772,99926,1445,1398
33773,99927,11,11


In [20]:
# convert all columns to numeric
hispanic_2021 = hispanic_2021.apply(pd.to_numeric)


In [21]:
hispanic_2021['hispanic_or_latino'] = hispanic_2021['total'] - hispanic_2021['not_hispanic_or_latino']
hispanic_2021['pct_hispanic_or_latino'] = hispanic_2021['hispanic_or_latino'] / hispanic_2021['total'] * 100
hispanic_2021

,zip,total,not_hispanic_or_latino,hispanic_or_latino,pct_hispanic_or_latino
1,601,17126,88,17038,99.486161
2,602,37895,2246,35649,94.073097
3,603,49136,1015,48121,97.934305
4,606,5751,41,5710,99.287081
5,610,26153,1100,25053,95.793982
...,...,...,...,...,...
33770,99923,13,13,0,0.000000
33771,99925,917,879,38,4.143948
33772,99926,1445,1398,47,3.252595
33773,99927,11,11,0,0.000000


In [22]:
# merge the two dataframes
racial_data_2021 = pd.merge(racial_2021, hispanic_2021, on='zip')
racial_data_2021


,zip,total_x,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander,pct_white,pct_black_or_african_american,pct_american_indian_or_alaska_native,pct_asian,pct_native_hawaiian_and_other_pacific_islander,total_y,not_hispanic_or_latino,hispanic_or_latino,pct_hispanic_or_latino
0,601,17126,14463,243,13,2,0,84.450543,1.418895,0.075908,0.011678,0.000000,17126,88,17038,99.486161
1,602,37895,23237,688,44,46,0,61.319435,1.815543,0.116110,0.121388,0.000000,37895,2246,35649,94.073097
2,603,49136,36497,1681,54,38,0,74.277515,3.421117,0.109899,0.077336,0.000000,49136,1015,48121,97.934305
3,606,5751,3319,27,21,0,0,57.711702,0.469484,0.365154,0.000000,0.000000,5751,41,5710,99.287081
4,610,26153,15815,642,0,0,0,60.471074,2.454785,0.000000,0.000000,0.000000,26153,1100,25053,95.793982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33769,99923,13,13,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,13,13,0,0.000000
33770,99925,917,411,4,335,7,0,44.820065,0.436205,36.532170,0.763359,0.000000,917,879,38,4.143948
33771,99926,1445,92,0,1304,6,0,6.366782,0.000000,90.242215,0.415225,0.000000,1445,1398,47,3.252595
33772,99927,11,11,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,11,11,0,0.000000


In [23]:
# remove columns
racial_data_2021 = racial_data_2021.drop(columns=['total_y', 'not_hispanic_or_latino'])
racial_data_2021 = racial_data_2021.rename(columns={'total_x': 'total_race'})
racial_data_2021

,zip,total_race,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander,pct_white,pct_black_or_african_american,pct_american_indian_or_alaska_native,pct_asian,pct_native_hawaiian_and_other_pacific_islander,hispanic_or_latino,pct_hispanic_or_latino
0,601,17126,14463,243,13,2,0,84.450543,1.418895,0.075908,0.011678,0.000000,17038,99.486161
1,602,37895,23237,688,44,46,0,61.319435,1.815543,0.116110,0.121388,0.000000,35649,94.073097
2,603,49136,36497,1681,54,38,0,74.277515,3.421117,0.109899,0.077336,0.000000,48121,97.934305
3,606,5751,3319,27,21,0,0,57.711702,0.469484,0.365154,0.000000,0.000000,5710,99.287081
4,610,26153,15815,642,0,0,0,60.471074,2.454785,0.000000,0.000000,0.000000,25053,95.793982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33769,99923,13,13,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000
33770,99925,917,411,4,335,7,0,44.820065,0.436205,36.532170,0.763359,0.000000,38,4.143948
33771,99926,1445,92,0,1304,6,0,6.366782,0.000000,90.242215,0.415225,0.000000,47,3.252595
33772,99927,11,11,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000


In [24]:
# rename columns 
racial_data_2021 = racial_data_2021.rename(columns={
    'total_race': 'total_race_2021',
    'white': 'white_2021',
    'black_or_african_american': 'black_or_african_american_2021',
    'american_indian_or_alaska_native': 'american_indian_or_alaska_native_2021',
    'asian': 'asian_2021',
    'native_hawaiian_and_other_pacific_islander': 'native_hawaiian_and_other_pacific_islander_2021',
    'pct_white': 'pct_white_2021',
    'pct_black_or_african_american': 'pct_black_or_african_american_2021',
    'pct_american_indian_or_alaska_native': 'pct_american_indian_or_alaska_native_2021',
    'pct_asian': 'pct_asian_2021',
    'pct_native_hawaiian_and_other_pacific_islander': 'pct_native_hawaiian_and_other_pacific_islander_2021',
    'hispanic_or_latino': 'hispanic_or_latino_2021',
    'pct_hispanic_or_latino': 'pct_hispanic_or_latino_2021'
})
racial_data_2021


,zip,total_race_2021,white_2021,black_or_african_american_2021,american_indian_or_alaska_native_2021,asian_2021,native_hawaiian_and_other_pacific_islander_2021,pct_white_2021,pct_black_or_african_american_2021,pct_american_indian_or_alaska_native_2021,pct_asian_2021,pct_native_hawaiian_and_other_pacific_islander_2021,hispanic_or_latino_2021,pct_hispanic_or_latino_2021
0,601,17126,14463,243,13,2,0,84.450543,1.418895,0.075908,0.011678,0.000000,17038,99.486161
1,602,37895,23237,688,44,46,0,61.319435,1.815543,0.116110,0.121388,0.000000,35649,94.073097
2,603,49136,36497,1681,54,38,0,74.277515,3.421117,0.109899,0.077336,0.000000,48121,97.934305
3,606,5751,3319,27,21,0,0,57.711702,0.469484,0.365154,0.000000,0.000000,5710,99.287081
4,610,26153,15815,642,0,0,0,60.471074,2.454785,0.000000,0.000000,0.000000,25053,95.793982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33769,99923,13,13,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000
33770,99925,917,411,4,335,7,0,44.820065,0.436205,36.532170,0.763359,0.000000,38,4.143948
33771,99926,1445,92,0,1304,6,0,6.366782,0.000000,90.242215,0.415225,0.000000,47,3.252595
33772,99927,11,11,0,0,0,0,100.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000


In [25]:
# get race data for 2020
racial_2020 = get_from_census("B02001", "2020")
racial_2020 = racial_2020[['zip code tabulation area', 'B02001_001E', 'B02001_002E', 'B02001_003E', 'B02001_004E', 'B02001_005E', 'B02001_006E']]
racial_2020 = racial_2020.rename(columns={
    'zip code tabulation area': 'zip',
    'B02001_001E': 'total',
    'B02001_002E': 'white',
    'B02001_003E': 'black_or_african_american',
    'B02001_004E': 'american_indian_or_alaska_native',
    'B02001_005E': 'asian',
    'B02001_006E': 'native_hawaiian_and_other_pacific_islander'
})
racial_2020 = racial_2020.apply(pd.to_numeric)
racial_2020





,zip,total,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander
1,29590,3543,718,2617,27,0,0
2,93306,74296,51869,1727,1208,2455,178
3,93660,4082,2231,40,0,0,0
4,93110,15777,12042,311,287,674,0
5,93212,22596,13033,2708,333,177,23
...,...,...,...,...,...,...,...
33116,16623,552,550,0,0,0,0
33117,16627,2118,1980,9,2,0,0
33118,16634,315,311,0,0,4,0
33119,16640,707,701,3,0,3,0


In [26]:
racial_2020['pct_white'] = racial_2020['white'] / racial_2020['total'] * 100
racial_2020['pct_black_or_african_american'] = racial_2020['black_or_african_american'] / racial_2020['total'] * 100
racial_2020['pct_american_indian_or_alaska_native'] = racial_2020['american_indian_or_alaska_native'] / racial_2020['total'] * 100
racial_2020['pct_asian'] = racial_2020['asian'] / racial_2020['total'] * 100
racial_2020['pct_native_hawaiian_and_other_pacific_islander'] = racial_2020['native_hawaiian_and_other_pacific_islander'] / racial_2020['total'] * 100
racial_2020

,zip,total,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander,pct_white,pct_black_or_african_american,pct_american_indian_or_alaska_native,pct_asian,pct_native_hawaiian_and_other_pacific_islander
1,29590,3543,718,2617,27,0,0,20.265312,73.863957,0.762066,0.000000,0.000000
2,93306,74296,51869,1727,1208,2455,178,69.813987,2.324486,1.625929,3.304350,0.239582
3,93660,4082,2231,40,0,0,0,54.654581,0.979912,0.000000,0.000000,0.000000
4,93110,15777,12042,311,287,674,0,76.326298,1.971224,1.819104,4.272042,0.000000
5,93212,22596,13033,2708,333,177,23,57.678350,11.984422,1.473712,0.783324,0.101788
...,...,...,...,...,...,...,...,...,...,...,...,...
33116,16623,552,550,0,0,0,0,99.637681,0.000000,0.000000,0.000000,0.000000
33117,16627,2118,1980,9,2,0,0,93.484419,0.424929,0.094429,0.000000,0.000000
33118,16634,315,311,0,0,4,0,98.730159,0.000000,0.000000,1.269841,0.000000
33119,16640,707,701,3,0,3,0,99.151344,0.424328,0.000000,0.424328,0.000000


In [27]:
# get hispanic data for 2020
hispanic_2020 = get_from_census("B03002", "2020")
hispanic_2020 = hispanic_2020[['zip code tabulation area', 'B03002_001E', 'B03002_002E']]
hispanic_2020 = hispanic_2020.rename(columns={
    'zip code tabulation area': 'zip',
})
hispanic_2020



,zip,B03002_001E,B03002_002E
1,29590,3543,3197
2,93306,74296,26180
3,93660,4082,157
4,93110,15777,11526
5,93212,22596,6955
...,...,...,...
33116,16623,552,552
33117,16627,2118,2114
33118,16634,315,315
33119,16640,707,702


In [28]:
hispanic_2020 = hispanic_2020.rename(columns={
    'B03002_001E': 'total',
    'B03002_002E': 'not_hispanic_or_latino',
})

hispanic_2020

,zip,total,not_hispanic_or_latino
1,29590,3543,3197
2,93306,74296,26180
3,93660,4082,157
4,93110,15777,11526
5,93212,22596,6955
...,...,...,...
33116,16623,552,552
33117,16627,2118,2114
33118,16634,315,315
33119,16640,707,702


In [29]:
hispanic_2020 = hispanic_2020.apply(pd.to_numeric)
hispanic_2020['hispanic_or_latino'] = hispanic_2020['total'] - hispanic_2020['not_hispanic_or_latino']
hispanic_2020['pct_hispanic_or_latino'] = hispanic_2020['hispanic_or_latino'] / hispanic_2020['total'] * 100
hispanic_2020

,zip,total,not_hispanic_or_latino,hispanic_or_latino,pct_hispanic_or_latino
1,29590,3543,3197,346,9.765735
2,93306,74296,26180,48116,64.762571
3,93660,4082,157,3925,96.153846
4,93110,15777,11526,4251,26.944286
5,93212,22596,6955,15641,69.220216
...,...,...,...,...,...
33116,16623,552,552,0,0.000000
33117,16627,2118,2114,4,0.188857
33118,16634,315,315,0,0.000000
33119,16640,707,702,5,0.707214


In [30]:
# merge the two dataframes
racial_data_2020 = pd.merge(racial_2020, hispanic_2020, on='zip')
racial_data_2020 = racial_data_2020.drop(columns=['total_y', 'not_hispanic_or_latino'])
racial_data_2020 = racial_data_2020.rename(columns={'total_x': 'total_race'})
racial_data_2020

,zip,total_race,white,black_or_african_american,american_indian_or_alaska_native,asian,native_hawaiian_and_other_pacific_islander,pct_white,pct_black_or_african_american,pct_american_indian_or_alaska_native,pct_asian,pct_native_hawaiian_and_other_pacific_islander,hispanic_or_latino,pct_hispanic_or_latino
0,29590,3543,718,2617,27,0,0,20.265312,73.863957,0.762066,0.000000,0.000000,346,9.765735
1,93306,74296,51869,1727,1208,2455,178,69.813987,2.324486,1.625929,3.304350,0.239582,48116,64.762571
2,93660,4082,2231,40,0,0,0,54.654581,0.979912,0.000000,0.000000,0.000000,3925,96.153846
3,93110,15777,12042,311,287,674,0,76.326298,1.971224,1.819104,4.272042,0.000000,4251,26.944286
4,93212,22596,13033,2708,333,177,23,57.678350,11.984422,1.473712,0.783324,0.101788,15641,69.220216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33115,16623,552,550,0,0,0,0,99.637681,0.000000,0.000000,0.000000,0.000000,0,0.000000
33116,16627,2118,1980,9,2,0,0,93.484419,0.424929,0.094429,0.000000,0.000000,4,0.188857
33117,16634,315,311,0,0,4,0,98.730159,0.000000,0.000000,1.269841,0.000000,0,0.000000
33118,16640,707,701,3,0,3,0,99.151344,0.424328,0.000000,0.424328,0.000000,5,0.707214


In [31]:
# rename columns 
racial_data_2020 = racial_data_2020.rename(columns={
    'total_race': 'total_race_2020',
    'white': 'white_2020',
    'black_or_african_american': 'black_or_african_american_2020',
    'american_indian_or_alaska_native': 'american_indian_or_alaska_native_2020',
    'asian': 'asian_2020',
    'native_hawaiian_and_other_pacific_islander': 'native_hawaiian_and_other_pacific_islander_2020',
    'pct_white': 'pct_white_2020',
    'pct_black_or_african_american': 'pct_black_or_african_american_2020',
    'pct_american_indian_or_alaska_native': 'pct_american_indian_or_alaska_native_2020',
    'pct_asian': 'pct_asian_2020',
    'pct_native_hawaiian_and_other_pacific_islander': 'pct_native_hawaiian_and_other_pacific_islander_2020',
    'hispanic_or_latino': 'hispanic_or_latino_2020',
    'pct_hispanic_or_latino': 'pct_hispanic_or_latino_2020'
})
racial_data_2020

,zip,total_race_2020,white_2020,black_or_african_american_2020,american_indian_or_alaska_native_2020,asian_2020,native_hawaiian_and_other_pacific_islander_2020,pct_white_2020,pct_black_or_african_american_2020,pct_american_indian_or_alaska_native_2020,pct_asian_2020,pct_native_hawaiian_and_other_pacific_islander_2020,hispanic_or_latino_2020,pct_hispanic_or_latino_2020
0,29590,3543,718,2617,27,0,0,20.265312,73.863957,0.762066,0.000000,0.000000,346,9.765735
1,93306,74296,51869,1727,1208,2455,178,69.813987,2.324486,1.625929,3.304350,0.239582,48116,64.762571
2,93660,4082,2231,40,0,0,0,54.654581,0.979912,0.000000,0.000000,0.000000,3925,96.153846
3,93110,15777,12042,311,287,674,0,76.326298,1.971224,1.819104,4.272042,0.000000,4251,26.944286
4,93212,22596,13033,2708,333,177,23,57.678350,11.984422,1.473712,0.783324,0.101788,15641,69.220216
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33115,16623,552,550,0,0,0,0,99.637681,0.000000,0.000000,0.000000,0.000000,0,0.000000
33116,16627,2118,1980,9,2,0,0,93.484419,0.424929,0.094429,0.000000,0.000000,4,0.188857
33117,16634,315,311,0,0,4,0,98.730159,0.000000,0.000000,1.269841,0.000000,0,0.000000
33118,16640,707,701,3,0,3,0,99.151344,0.424328,0.000000,0.424328,0.000000,5,0.707214


In [32]:
# merge racial data for 2020 and 2021

racial_data = pd.merge(racial_data_2020, racial_data_2021, on='zip')
racial_data

,zip,total_race_2020,white_2020,black_or_african_american_2020,american_indian_or_alaska_native_2020,asian_2020,native_hawaiian_and_other_pacific_islander_2020,pct_white_2020,pct_black_or_african_american_2020,pct_american_indian_or_alaska_native_2020,...,american_indian_or_alaska_native_2021,asian_2021,native_hawaiian_and_other_pacific_islander_2021,pct_white_2021,pct_black_or_african_american_2021,pct_american_indian_or_alaska_native_2021,pct_asian_2021,pct_native_hawaiian_and_other_pacific_islander_2021,hispanic_or_latino_2021,pct_hispanic_or_latino_2021
0,29590,3543,718,2617,27,0,0,20.265312,73.863957,0.762066,...,25,14,0,23.286517,72.584270,0.702247,0.393258,0.000000,299,8.398876
1,93306,74296,51869,1727,1208,2455,178,69.813987,2.324486,1.625929,...,1204,2447,117,61.578411,3.320003,1.615717,3.283770,0.157009,49268,66.115569
2,93660,4082,2231,40,0,0,0,54.654581,0.979912,0.000000,...,28,0,0,44.589650,0.967067,0.731835,0.000000,0.000000,3676,96.079456
3,93110,15777,12042,311,287,674,0,76.326298,1.971224,1.819104,...,210,863,6,73.049862,1.211380,1.284797,5.279902,0.036708,4481,27.415112
4,93212,22596,13033,2708,333,177,23,57.678350,11.984422,1.473712,...,500,237,42,47.464144,13.772248,2.160014,1.023847,0.181441,16159,69.807327
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32918,16623,552,550,0,0,0,0,99.637681,0.000000,0.000000,...,0,0,0,99.823944,0.000000,0.000000,0.000000,0.000000,0,0.000000
32919,16627,2118,1980,9,2,0,0,93.484419,0.424929,0.094429,...,1,0,1,88.642413,0.532387,0.044366,0.000000,0.044366,13,0.576752
32920,16634,315,311,0,0,4,0,98.730159,0.000000,0.000000,...,0,11,0,93.818182,0.000000,0.000000,4.000000,0.000000,0,0.000000
32921,16640,707,701,3,0,3,0,99.151344,0.424328,0.000000,...,0,2,0,98.757062,0.225989,0.000000,0.225989,0.000000,2,0.225989


In [34]:
# save to csv
racial_data.to_csv('racial_data.csv', index=False)

# Percenatge below poverty

In [ ]:
# get poverty data for 2021
poverty_2021 = get_from_census("B17017", "2021")

# clean it up
poverty_2021 = poverty_2021[['zip code tabulation area', 'B17017_002E', 'B17017_001E']]
poverty_2021.rename(columns={
    'zip code tabulation area': 'zip',
})

,zip,B17017_002E,B17017_001E
1,00601,3520,5397
2,00602,6300,12858
3,00603,10047,19295
4,00606,957,1968
5,00610,4222,8934
...,...,...,...
33770,99923,0,13
33771,99925,63,345
33772,99926,55,457
33773,99927,0,11


In [ ]:
# rename columns
poverty_2021.rename(columns={'B17017_002E': 'households_below_poverty', 'B17017_001E': 'total_households', 'zip code tabulation area': 'zip'}, inplace=True)



In [ ]:
poverty_2021

,zip,households_below_poverty,total_households
1,00601,3520,5397
2,00602,6300,12858
3,00603,10047,19295
4,00606,957,1968
5,00610,4222,8934
...,...,...,...
33770,99923,0,13
33771,99925,63,345
33772,99926,55,457
33773,99927,0,11


In [ ]:
poverty_2021['pct_below_poverty_2021'] = poverty_2021['households_below_poverty'].astype(int) / poverty_2021['total_households'].astype(int) * 100
poverty_2021

,zip,households_below_poverty,total_households,pct_below_poverty_2021
1,00601,3520,5397,65.221419
2,00602,6300,12858,48.996734
3,00603,10047,19295,52.070485
4,00606,957,1968,48.628049
5,00610,4222,8934,47.257667
...,...,...,...,...
33770,99923,0,13,0.000000
33771,99925,63,345,18.260870
33772,99926,55,457,12.035011
33773,99927,0,11,0.000000


In [ ]:
# get poverty data for 2020 
poverty_2020 = get_from_census("B17017", "2020")

# clean it up
poverty_2020 = poverty_2020[['zip code tabulation area', 'B17017_002E', 'B17017_001E']]
poverty_2020.rename(columns={
    'zip code tabulation area': 'zip',
})

,zip,B17017_002E,B17017_001E
1,29590,333,986
2,93306,4006,21974
3,93660,288,949
4,93110,749,6385
5,93212,1074,4014
...,...,...,...
33116,16623,29,216
33117,16627,138,880
33118,16634,14,146
33119,16640,49,299


In [ ]:
poverty_2020.rename(columns={'B17017_002E': 'households_below_poverty', 'B17017_001E': 'total_households', 'zip code tabulation area': 'zip'}, inplace=True)


In [ ]:
poverty_2020['pct_below_poverty_2020'] = poverty_2020['households_below_poverty'].astype(int) / poverty_2020['total_households'].astype(int) * 100
poverty_2020

,zip,households_below_poverty,total_households,pct_below_poverty_2020
1,29590,333,986,33.772819
2,93306,4006,21974,18.230636
3,93660,288,949,30.347734
4,93110,749,6385,11.730619
5,93212,1074,4014,26.756353
...,...,...,...,...
33116,16623,29,216,13.425926
33117,16627,138,880,15.681818
33118,16634,14,146,9.589041
33119,16640,49,299,16.387960


In [ ]:
pct_below_poverty = pd.merge(poverty_2020, poverty_2021, on='zip', how='outer')
pct_below_poverty

,zip,households_below_poverty_x,total_households_x,pct_below_poverty_2020,households_below_poverty_y,total_households_y,pct_below_poverty_2021
0,29590,333,986,33.772819,309,908,34.030837
1,93306,4006,21974,18.230636,3918,21818,17.957650
2,93660,288,949,30.347734,291,868,33.525346
3,93110,749,6385,11.730619,670,6474,10.349089
4,93212,1074,4014,26.756353,1112,4230,26.288416
...,...,...,...,...,...,...,...
33966,99635,NaN,NaN,NaN,0,0,NaN
33967,99675,NaN,NaN,NaN,6,11,54.545455
33968,99707,NaN,NaN,NaN,0,0,NaN
33969,99725,NaN,NaN,NaN,57,314,18.152866


In [ ]:
# drop columns

pct_below_poverty.drop(columns=['households_below_poverty_x', 'total_households_x', 'households_below_poverty_y', 'total_households_y'], inplace=True)



In [ ]:
# rename columns

pct_below_poverty.rename(columns={'pct_below_poverty_2020_x': 'pct_below_poverty_2020', 'pct_below_poverty_2020_y': 'pct_below_poverty_2021'}, inplace=True)
pct_below_poverty

,zip,pct_below_poverty_2020,pct_below_poverty_2021
0,29590,33.772819,34.030837
1,93306,18.230636,17.957650
2,93660,30.347734,33.525346
3,93110,11.730619,10.349089
4,93212,26.756353,26.288416
...,...,...,...
33966,99635,NaN,NaN
33967,99675,NaN,54.545455
33968,99707,NaN,NaN
33969,99725,NaN,18.152866


In [ ]:
# save to csv

# pct_below_poverty.to_csv('pct_below_poverty.csv', index=False)

# Median income 

In [ ]:
# get income data for 2021
median_income_2021 = get_from_census("B19013", "2021")

# clean it up
median_income_2021 = median_income_2021[['zip code tabulation area', 'B19013_001E']]
median_income_2021.rename(columns={
    'zip code tabulation area': 'zip',
})

,zip,B19013_001E
1,00601,15292
2,00602,18716
3,00603,16789
4,00606,18835
5,00610,21239
...,...,...
33770,99923,-666666666
33771,99925,70625
33772,99926,58229
33773,99927,-666666666


In [ ]:
# create a column for the year
median_income_2021['year'] = 2021
median_income_2021

,zip code tabulation area,B19013_001E,year
1,00601,15292,2021
2,00602,18716,2021
3,00603,16789,2021
4,00606,18835,2021
5,00610,21239,2021
...,...,...,...
33770,99923,-666666666,2021
33771,99925,70625,2021
33772,99926,58229,2021
33773,99927,-666666666,2021


In [ ]:
# rename zip code tabulation area to zip    
median_income_2021.rename(columns={'zip code tabulation area': 'zip', 'B19013_001E': 'med_inc_2021'}, inplace=True)
median_income_2021

,zip,med_inc_2021,year
1,00601,15292,2021
2,00602,18716,2021
3,00603,16789,2021
4,00606,18835,2021
5,00610,21239,2021
...,...,...,...
33770,99923,-666666666,2021
33771,99925,70625,2021
33772,99926,58229,2021
33773,99927,-666666666,2021


In [ ]:
# get income data for 2020
median_income_2020 = get_from_census("B19013", "2020")

# clean it up
median_income_2020 = median_income_2020[['zip code tabulation area', 'B19013_001E']]
median_income_2020.rename(columns={
    'zip code tabulation area': 'zip',
})

,zip,B19013_001E
1,29590,30985
2,93306,54450
3,93660,39625
4,93110,93264
5,93212,42983
...,...,...
33116,16623,51667
33117,16627,45000
33118,16634,51500
33119,16640,55982


In [ ]:
# create a column for the year
median_income_2020['year'] = 2020
median_income_2020

,zip code tabulation area,B19013_001E,year
1,29590,30985,2020
2,93306,54450,2020
3,93660,39625,2020
4,93110,93264,2020
5,93212,42983,2020
...,...,...,...
33116,16623,51667,2020
33117,16627,45000,2020
33118,16634,51500,2020
33119,16640,55982,2020


In [ ]:
# rename zip code tabulation area to zip    
median_income_2020.rename(columns={'zip code tabulation area': 'zip', 'B19013_001E': 'med_inc_2020'}, inplace=True)
median_income_2020

,zip,med_inc_2020,year
1,29590,30985,2020
2,93306,54450,2020
3,93660,39625,2020
4,93110,93264,2020
5,93212,42983,2020
...,...,...,...
33116,16623,51667,2020
33117,16627,45000,2020
33118,16634,51500,2020
33119,16640,55982,2020


In [ ]:
median_income = pd.merge(median_income_2020, median_income_2021, on='zip', how='outer')
median_income

,zip,med_inc_2020,year_x,med_inc_2021,year_y
0,29590,30985,2020.0,52679,2021.0
1,93306,54450,2020.0,60857,2021.0
2,93660,39625,2020.0,40000,2021.0
3,93110,93264,2020.0,99261,2021.0
4,93212,42983,2020.0,46312,2021.0
...,...,...,...,...,...
33966,99635,NaN,NaN,-666666666,2021.0
33967,99675,NaN,NaN,34375,2021.0
33968,99707,NaN,NaN,-666666666,2021.0
33969,99725,NaN,NaN,45931,2021.0


In [ ]:
# drop columns year_x and year_y

median_income.drop(columns=['year_x', 'year_y'], inplace=True)
median_income

,zip,med_inc_2020,med_inc_2021
0,29590,30985,52679
1,93306,54450,60857
2,93660,39625,40000
3,93110,93264,99261
4,93212,42983,46312
...,...,...,...
33966,99635,NaN,-666666666
33967,99675,NaN,34375
33968,99707,NaN,-666666666
33969,99725,NaN,45931


# Population

In [ ]:
# get population data for 2021
population_2021 = get_from_census("B01003", "2021")

# clean it up
population_2021 = population_2021\
        [['zip code tabulation area', 'B01003_001E']]\
        .rename(columns={'zip code tabulation area': 'zip'})

population_2021

,zip,B01003_001E
1,00601,17126
2,00602,37895
3,00603,49136
4,00606,5751
5,00610,26153
...,...,...
33770,99923,13
33771,99925,917
33772,99926,1445
33773,99927,11


In [ ]:
#rename columns
population_2021.rename(columns={'B01003_001E': 'pop_2021'}, inplace=True)
population_2021

,zip,pop_2021
1,00601,17126
2,00602,37895
3,00603,49136
4,00606,5751
5,00610,26153
...,...,...
33770,99923,13
33771,99925,917
33772,99926,1445
33773,99927,11


In [ ]:
# get population data for 2020
population_2020 = get_from_census("B01003", "2020")

# clean it up
population_2020 = population_2020\
        [['zip code tabulation area', 'B01003_001E']]\
        .rename(columns={'zip code tabulation area': 'zip'})

population_2020

,zip,B01003_001E
1,29590,3543
2,93306,74296
3,93660,4082
4,93110,15777
5,93212,22596
...,...,...
33116,16623,552
33117,16627,2118
33118,16634,315
33119,16640,707


In [ ]:
#rename columns
population_2020.rename(columns={'B01003_001E': 'pop_2020'}, inplace=True)
population_2020

,zip,pop_2020
1,29590,3543
2,93306,74296
3,93660,4082
4,93110,15777
5,93212,22596
...,...,...
33116,16623,552
33117,16627,2118
33118,16634,315
33119,16640,707


In [ ]:
population = pd.merge(population_2020, population_2021, on='zip', how='outer')
population

,zip,pop_2020,pop_2021
0,29590,3543,3560
1,93306,74296,74518
2,93660,4082,3826
3,93110,15777,16345
4,93212,22596,23148
...,...,...,...
33966,99635,NaN,107
33967,99675,NaN,58
33968,99707,NaN,0
33969,99725,NaN,611


In [ ]:
# merge population and median income data into one dataframe

census_data = pd.merge(population, median_income, on='zip', how='inner')
census_data


,zip,pop_2020,pop_2021,med_inc_2020,med_inc_2021
0,29590,3543,3560,30985,52679
1,93306,74296,74518,54450,60857
2,93660,4082,3826,39625,40000
3,93110,15777,16345,93264,99261
4,93212,22596,23148,42983,46312
...,...,...,...,...,...
33966,99635,NaN,107,NaN,-666666666
33967,99675,NaN,58,NaN,34375
33968,99707,NaN,0,NaN,-666666666
33969,99725,NaN,611,NaN,45931


Merge with zip code csv

In [ ]:
# read csv
zip_codes = pd.read_csv('nyc_zip_codes_cleaned.csv')
zip_codes

FileNotFoundError: [Errno 2] No such file or directory: 'nyc_zip_codes_cleaned.csv'

In [ ]:
zip_codes.dtypes

zip         int64
borough    object
dtype: object

In [ ]:
census_data.dtypes


zip             object
pop_2020        object
pop_2021        object
med_inc_2020    object
med_inc_2021    object
dtype: object

In [ ]:
# convert zip codes to int

census_data['zip'] = census_data['zip'].astype(int)
census_data.dtypes


zip              int64
pop_2020        object
pop_2021        object
med_inc_2020    object
med_inc_2021    object
dtype: object

In [ ]:
# left join zip codes with census data

census = pd.merge(zip_codes, census_data, on='zip', how='left')
census

,zip,borough,pop_2020,pop_2021,med_inc_2020,med_inc_2021
0,10001,Manhattan,25026,26966,96787,101409
1,10002,Manhattan,74363,76807,35607,37093
2,10003,Manhattan,54671,54447,129981,137533
3,10004,Manhattan,3310,4795,204949,216017
4,10005,Manhattan,8664,8637,184681,197188
...,...,...,...,...,...,...
475,10309,Staten,33896,35832,102730,107500
476,10310,Staten,24168,25976,86895,96161
477,10311,Staten,0,0,-666666666,-666666666
478,10312,Staten,61114,63935,96785,100875


In [ ]:
# data for 2022 is not available yet, so we will use 2021 data for 2022
census['pop_2022'] = census['pop_2021']
census['med_inc_2022'] = census['med_inc_2021']
census

,zip,borough,pop_2020,pop_2021,med_inc_2020,med_inc_2021,pop_2022,med_inc_2022
0,10001,Manhattan,25026,26966,96787,101409,26966,101409
1,10002,Manhattan,74363,76807,35607,37093,76807,37093
2,10003,Manhattan,54671,54447,129981,137533,54447,137533
3,10004,Manhattan,3310,4795,204949,216017,4795,216017
4,10005,Manhattan,8664,8637,184681,197188,8637,197188
...,...,...,...,...,...,...,...,...
475,10309,Staten,33896,35832,102730,107500,35832,107500
476,10310,Staten,24168,25976,86895,96161,25976,96161
477,10311,Staten,0,0,-666666666,-666666666,0,-666666666
478,10312,Staten,61114,63935,96785,100875,63935,100875


In [ ]:
# how many unique zip codes are there?

census['zip'].nunique()

240

In [ ]:
# save to csv
# census.to_csv('census_data.csv', index=False)

### Block level data from Census API

In [41]:
# df = pd.read_csv('address_level.csv')
# df


,year,borough,zip,incident_address,lat,long,num_complaints
0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1
1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108
2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2
3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2
4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2
...,...,...,...,...,...,...,...
106127,2022,STATEN ISLAND,10314,81 CHURCH AVENUE,40.592844,-74.185280,1
106128,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587616,-74.165297,1
106129,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587651,-74.165359,5
106130,2022,STATEN ISLAND,10314,98 NOSTRAND AVENUE,40.606914,-74.169706,1


In [43]:
# save as parquet

# df.to_parquet('address_level.parquet', index=False)

In [44]:
df_address = pd.read_parquet('address_level.parquet')
df_address

,year,borough,zip,incident_address,lat,long,num_complaints
0,2020,BRONX,10451,1020 GRAND CONCOURSE,40.830314,-73.920785,1
1,2020,BRONX,10451,109 EAST 153 STREET,40.823932,-73.928033,108
2,2020,BRONX,10451,175 EAST 151 STREET,40.820882,-73.927299,2
3,2020,BRONX,10451,180 EAST 162 STREET,40.827449,-73.921442,2
4,2020,BRONX,10451,180 EAST 163 STREET,40.828272,-73.920754,2
...,...,...,...,...,...,...,...
106127,2022,STATEN ISLAND,10314,81 CHURCH AVENUE,40.592844,-74.185280,1
106128,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587616,-74.165297,1
106129,2022,STATEN ISLAND,10314,90 RICHMOND HILL ROAD,40.587651,-74.165359,5
106130,2022,STATEN ISLAND,10314,98 NOSTRAND AVENUE,40.606914,-74.169706,1


In [51]:
def geocode(lat, lng):
    census = cg.coordinates(lng, lat)['2020 Census Blocks'][0]

    data = dict(geoid=census['GEOID'], 
                state=census['STATE'], 
                county=census['COUNTY'], 
                tract=census['TRACT'], 
                block=census['BLOCK'])
    
    return data


with ThreadPoolExecutor() as tpe:
    latitudes = df_address['lat']
    longitudes = df_address['long']
    mapped_results = tpe.map(geocode, latitudes, longitudes)
    data = list(tqdm(mapped_results, total=len(df_address)))

census_geos_df = pd.DataFrame(data)
census_geos_df.head()

  0%|          | 0/106132 [00:00<?, ?it/s]